In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import re
import os
from bs4 import BeautifulSoup

In [2]:
DEPARTMENT_KEYWORD = 'STAT'

In [3]:
data_directory = os.getcwd()+'/data/'
data_directory

'/Users/mafuangimemkamon/Documents/GitHub/harvard_syllabi_topic_model/data/'

In [4]:
chromeOptions = webdriver.ChromeOptions()
prefs = {"download.default_directory" : data_directory}
chromeOptions.add_experimental_option("prefs",prefs)

In [5]:
driver = webdriver.Chrome(options=chromeOptions)
driver.get("https://syllabus.harvard.edu/search/courses/?q={}".format(DEPARTMENT_KEYWORD))

In [6]:
def count_results(driver):
    result_text = page_html = driver.page_source
    match = re.search(r"Loaded (\d+) results", result_text)
    if match:
        number_of_results = int(match.group(1))
        return number_of_results
    else:
        return 0

In [7]:
# Click expand more button until can't click anymore
num = 0
new_num = count_results(driver)
while new_num > num:
    time.sleep(0.5)
    try:
        button = driver.find_element("css selector", "button.css-jrdf8z-view--inlineBlock-baseButton")
        button.click()
    except:
        pass
    time.sleep(1)
    num = new_num
    new_num = count_results(driver)
    print(new_num, num, end='\r')

In [8]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
names = [i.text for i in soup.select('.css-32gbjx-text')][1:]
ids = [i.get('id') for i in soup.find_all('tr')][1:]
assert len(names) == len(ids)
print(len(names))

287


In [20]:
import pandas as pd
adf = pd.DataFrame(names, columns = ['Name'])
adf['code'] = adf['Name'].str.split(':').str[0]
adf.to_csv('processed_data/stat_mapping.csv', index = False)

In [9]:
for i,id in enumerate(ids):
    print(i, names[i])
    url = "https://syllabus.harvard.edu/search/courses/?c={}&q={}".format(id, DEPARTMENT_KEYWORD)
    driver.get(url)
    time.sleep(1.5)
    try:
        download_button = driver.find_element("css selector", "button#download-btn")
        download_button.click()
    except:
        pass
    time.sleep(1.5)
driver.quit()

STAT 213: Statistical Inference II
STAT 213: Statistical Inference 2
STAT 109: Intro to Statistical Modelling
EDU S043/STAT 151: Multilevel and Longitudinal Models
BIOSTAT 282/BST 282/STAT 115/STAT 215: Introduction to Computational Biology and Bioinformatics
STAT 111: Introduction to Statistical Inference
STAT 100: Introduction to Statistics and Data Science
STAT 139: Introduction to Linear Models
STAT 100: Introduction to Quantitative Methods for the Social Sciences and Humanities
STAT 139: Statistical Sleuthing Through Linear Models
STAT 139: Linear Models
STAT 131: Time Series & Prediction
STAT 230: Multivariate Statistical Analysis
STAT 160: Introduction to Survey Sampling and Estimation
STAT 186: Statistical Methods for Evaluating Causal Effects
ECON 2142/STAT 242: Time Series
STAT 160/STAT 260: Design and Analysis of Sample Surveys
STAT 120: Introduction to Bayesian Inference and Applications
STAT 123: Applied Quantitative Finance
APCOMP 209A/COMPSCI 1090A/COMPSCI 109A/STAT 109A